## 수정 사항
1. 백업 스토어 설정에서 InMemoryDocstore를 사용해보기
2. 파라미터 조정
3. 프롬프트에 명령 추가



## 백터 스토어 설정에서 InMemoryDocstore를 사용해보기

In [ ]:
# 벡터 스토어 생성
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

vector_dim = len(embeddings.embed_query("example text")) 
index = faiss.IndexFlatL2(vector_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

## 파라미터 조정

- 모델 지정에서 파라미터 추가(temperature, max_tokens)
- 리트리버 변환에서 k값 변경(1->2)
- 모델 로드에 UTF-8 추가


### 리트리버 파라미터 설명
- search_type
 - similarity: 벡터간 유사도, 내적 사용 검색
 - mmr: 다양성 고려 검색
 - approx: 근사적 검색(대규모 데이터셋에서 빠른 검색)

- search_kwargs
 - k: 반환할 결과의 수 
 - lambda_mult: 유사도와 다양성 간의 균형 조정

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.3, max_tokens = 1000)


In [ ]:
# 파일 로드
from langchain.document_loaders import PyPDFLoader

pdf_path = "초거대언어모델연구동향.pdf"

loader = PyPDFLoader(pdf_path)

docs = loader.load()

for doc in docs:
    utf8_docs = [doc.page_content.encode('utf-8').decode('utf-8') for doc in docs]
    
    


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

## 프롬프트에 추가 명령

- system: 인공지능 모델 연구자이십니다. 이 문서에서는 발전, 과제 및 주요 기여를 포함하여 초대형 언어 모델의 최신 연구 동향을 설명합니다. 귀하의 임무는 문서를 분석하고 상세하고 통찰력 있는 답변을 제공하는 것입니다.
- user: 본 문서에 언급된 주요 동향과 연구 결과를 중심으로 포괄적인 답변을 부탁드립니다.


In [ ]:
# 템플릿 정의
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an artificial intelligence model researcher. The document describes the latest research trends in super-large language models, including advancements, challenges, and key contributions. Your task is to analyze the document and provide a detailed and insightful response."),
    ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer: Please provide a comprehensive response, focusing on the key trends and research findings mentioned in the document.")
])


# 수정 본

In [7]:
# KEY설정
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

# 모델로드
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.3, max_tokens = 1000)

# 파일 로드
from langchain.document_loaders import PyPDFLoader

pdf_path = "초거대언어모델연구동향.pdf"

loader = PyPDFLoader(pdf_path)

docs = loader.load()

for doc in docs:
    utf8_docs = [doc.page_content.encode('utf-8').decode('utf-8') for doc in docs]

# 문서 청크 나누기 2
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

    
# 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

# 벡터 스토어 생성
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

vector_dim = len(embeddings.embed_query("example text")) 
index = faiss.IndexFlatL2(vector_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# 리트리버 변환

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 템플릿 정의
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an artificial intelligence model researcher. The document describes the latest research trends in super-large language models, including advancements, challenges, and key contributions. Your task is to analyze the document and provide a detailed and insightful response."),
    ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer: Please provide a comprehensive response, focusing on the key trends and research findings mentioned in the document.")
])

# 질문 응답 체인 구성
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# 질문 반복 처리
rag_chain_debug = {
    "context": retriever,   # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()     # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()| contextual_prompt | model

# 출력
while True:
    print('=====================')
    query = input("질문을 입력하세요: ")

    if query.lower() == "exit":
        print("대답: 즐거운 대화였어요.")
        break

    response = rag_chain_debug.invoke(query)
    
    print("Final Response: ")
    print(response.content)

OpenAI API key 입력: ········
질문을 입력하세요: 최근 초거대 언어모델 연구동향을 알려줘
Debug Output: 최근 초거대 언어모델 연구동향을 알려줘
Debug Output: {'context': [], 'question': '최근 초거대 언어모델 연구동향을 알려줘'}
Final Response: 
최근 초거대 언어모델(LLM) 연구동향은 여러 가지 중요한 발전과 도전 과제를 포함하고 있습니다. 다음은 주요 트렌드와 연구 결과에 대한 종합적인 분석입니다.

1. **모델 크기와 성능**: 초거대 언어모델의 크기는 계속해서 증가하고 있으며, 이는 모델의 성능 향상에 기여하고 있습니다. 많은 연구자들이 파라미터 수를 늘리는 것뿐만 아니라, 더 정교한 아키텍처를 개발하여 모델의 이해력과 생성 능력을 개선하고 있습니다. 예를 들어, 최근의 연구에서는 수백억 개의 파라미터를 가진 모델들이 다양한 자연어 처리(NLP) 작업에서 이전 모델들보다 우수한 성능을 보이고 있습니다.

2. **효율성 개선**: 모델의 크기가 커짐에 따라 계산 자원과 에너지 소비에 대한 우려도 커지고 있습니다. 이에 따라 연구자들은 모델의 효율성을 높이기 위한 다양한 방법을 모색하고 있습니다. 예를 들어, 지식 증류(knowledge distillation), 프루닝(pruning), 양자화(quantization) 등의 기법이 활용되어, 성능을 유지하면서도 더 적은 자원으로 모델을 운영할 수 있는 방안이 연구되고 있습니다.

3. **다양한 활용 사례**: 초거대 언어모델은 다양한 분야에서 활용되고 있습니다. 특히, 고객 서비스, 콘텐츠 생성, 코드 작성, 번역 등 여러 산업에서 실제 적용 사례가 증가하고 있습니다. 이러한 활용은 모델의 상용화 가능성을 높이고 있으며, 기업들이 LLM을 통해 비즈니스 가치를 창출하는 데 기여하고 있습니다.

4. **윤리적 고려와 편향 문제**: LLM의 발전과 함께 윤리적 문제와 편향(bias) 문제도 중요한 연구 주제로 떠오르고 있습니

# 1.
- 질문: 최근 초거대 언어모델 연구동향을 알려줘
- 대답: 깔끔하게 정리 되었고 내용에 맞는 대답을 한다.

# 2. 
- 질문: 각 주제들 중에 가장 중요하다고 생각드는건 무엇이며 왜그런지 설명해주세요
- 대답: 갑자기 영어로 대답을하여 놀랐다. 내용은 초대형 언어 모델 배포의 윤리적 영향과 사회적 영향을 말하고있다.

# 3.
- 질문: 방금전 글을 한글로 말해줘
- 대답: 첫 번째 질문을 그대로 가져왔다. 

# 4.
- 질문: 각 주제들 중에 가장 중요하다고 생각드는건 무엇이며 왜그런지 한글로 설명해주세요 (한글로 설명 부탁)
- 대답: 질문에 관련없는 내용이 나옴


## 프롬프트에서 바꾸어야할 점

### 이전 프롬프트
- system: 인공지능 모델 연구자이십니다. 이 문서에서는 발전, 과제 및 주요 기여를 포함하여 초대형 언어 모델의 최신 연구 동향을 설명합니다. 귀하의 임무는 문서를 분석하고 상세하고 통찰력 있는 답변을 제공하는 것입니다.
- user: 본 문서에 언급된 주요 동향과 연구 결과를 중심으로 포괄적인 답변을 부탁드립니다.